## TYTANチュートリアル（ナップサック問題）

2023年5月31日

ビネクラ安田

出典：[【ラスボス】量子アニーリング（QUBO）でナップサック問題を解く](https://vigne-cla.com/21-27/)

### 問題
え、こんなに大変だったの？ナップサック問題。しかも欲張ってちょっと複雑な「重複を許して取れる」パターンの問題を用意。これが分かれば普通のナップサック問題も分かるし、QUBOマスターも近いはず<sup>[要出典]</sup>

**＜問題＞**<br>
**所持金750円でおいしいみず、サイコソーダ、ミックスオレを買うとき、回復量がもっとも多くなる組み合わせを求めよ。**ただし重複購入OKとする。

<div align="center">
<img src="https://vigne-cla.com/wp-content/uploads/2023/05/21-27_1r.png" width = 35%>
</div>

### 必要な知識（おさらい）

予習が重たすぎるのでとりあえず特攻しますか・・・

**応用C2「すべての量子ビットを降順にする」**

→ [1, 1, 0, 0] のように降順に絞ることができる

**応用D3　解が0と1だけの不等式**

→ 「1x + 2y + 3z ≦ 3」が解ける

これらに加えて、**「報酬ができるだけ多くなるように」**というコストの考え方、**条件間の重み付け**も用いる。

途中で理解が追いつかなくなったらチートページで予習を → [量子アニーリングのQUBOで設定可能な条件式まとめ（保存版）](https://vigne-cla.com/21-12/)

## 前半戦：750円ピッタリで買う場合

<font color="red">まず、全体像を掴むために「750円ピッタリで買う」バージョンを解く。</font>

**＜準備１＞**<br>
まず問題の数字を簡単にする。金額は50で割り、回復量は10で割る。数字のオーダーをだいたい揃えておくことで条件間の重みバランスの設定が簡単になったり、特に所持金を750から15にしたことで後半戦で出てくる不等式が考えやすくなる。

<div align="center">
<img src="https://vigne-cla.com/wp-content/uploads/2023/05/21-27_2r.png" width = 35%>
</div>

**＜準備２＞**<br>
重複を許して購入するため、おいしいみずは最大３個、サイコソーダは２個、ミックスオレは２個に分身させて量子ビットを割り当てる。１になったら買う、０になったら買わない。

<div align="center">
<img src="https://vigne-cla.com/wp-content/uploads/2023/05/21-27_3r.png" width = 40%>
</div>

**＜条件式１（強い条件）＞**<br>
７個のドリンクをそれぞれ取るか取らないかして、合計金額が15になる設定をする。取るは１、取らないは０であり、１のときに金額の係数が有効になってその合計が15になる。
```
H += (4*q0 + 4*q1 + 4*q2 + 6*q3 + 6*q4 + 7*q5 + 7*q6 - 15)**2
```

**＜条件式２（弱い条件）＞**<br>
あとは「回復量ができるだけ多くなるように」を設定する。７個のドリンクをそれぞれ取るか取らないかして、合計回復量を報酬にする。報酬なので全体にかかる係数はマイナス、「できるだけ～になるように」は弱い条件、ということで-0.1をかける。
```
H += -0.1 * (3*q0 + 3*q1 + 3*q2 + 5*q3 + 5*q4 + 7*q5 + 7*q6)
```

### 前半戦：コード

In [ ]:
!pip install git+https://github.com/tytansdk/tytan

In [4]:
from tytan import symbols, Compile, sampler
import numpy as np

#量子ビットを用意する
q0 = symbols('q0')
q1 = symbols('q1')
q2 = symbols('q2')
q3 = symbols('q3')
q4 = symbols('q4')
q5 = symbols('q5')
q6 = symbols('q6')

#7個のドリンクをそれぞれ取るか取らないか、値段を係数にして合計が15になる（強い条件）
H = 0
H += (4*q0 + 4*q1 + 4*q2 + 6*q3 + 6*q4 + 7*q5 + 7*q6 - 15)**2

#7個のドリンクをそれぞれ取るか取らないか、回復量を係数にして報酬とする（弱い条件）
H += -0.1 * (3*q0 + 3*q1 + 3*q2 + 5*q3 + 5*q4 + 7*q5 + 7*q6)


#コンパイル
qubo, offset = Compile(H).get_qubo()
print(f'offset\n{offset}')

#サンプラー選択
solver = sampler.SASampler()

#サンプリング
result = solver.run(qubo, shots=1000)

#上位5件
for r in result:
    print(r)

offset
225
[{'q0': 0.0, 'q1': 1.0, 'q2': 1.0, 'q3': 0.0, 'q4': 0.0, 'q5': 0.0, 'q6': 1.0}, -226.29999999999998, 181]
[{'q0': 0.0, 'q1': 1.0, 'q2': 1.0, 'q3': 0.0, 'q4': 0.0, 'q5': 1.0, 'q6': 0.0}, -226.29999999999998, 57]
[{'q0': 1.0, 'q1': 0.0, 'q2': 1.0, 'q3': 0.0, 'q4': 0.0, 'q5': 0.0, 'q6': 1.0}, -226.29999999999998, 174]
[{'q0': 1.0, 'q1': 0.0, 'q2': 1.0, 'q3': 0.0, 'q4': 0.0, 'q5': 1.0, 'q6': 0.0}, -226.29999999999998, 167]
[{'q0': 1.0, 'q1': 1.0, 'q2': 0.0, 'q3': 0.0, 'q4': 0.0, 'q5': 0.0, 'q6': 1.0}, -226.29999999999998, 129]
[{'q0': 1.0, 'q1': 1.0, 'q2': 0.0, 'q3': 0.0, 'q4': 0.0, 'q5': 1.0, 'q6': 0.0}, -226.29999999999998, 180]
[{'q0': 0.0, 'q1': 0.0, 'q2': 1.0, 'q3': 1.0, 'q4': 1.0, 'q5': 0.0, 'q6': 0.0}, -225.3, 15]
[{'q0': 0.0, 'q1': 1.0, 'q2': 0.0, 'q3': 1.0, 'q4': 1.0, 'q5': 0.0, 'q6': 0.0}, -225.3, 54]
[{'q0': 1.0, 'q1': 0.0, 'q2': 0.0, 'q3': 1.0, 'q4': 1.0, 'q5': 0.0, 'q6': 0.0}, -225.3, 43]


ベストな解が６通り出たが実質１通りで、**みず２本、ソーダ０本、オレ１本**である。マイナスオフセットより1.3低いエネルギーなので報酬が1.3、つまり回復量13と分かる。

果たしてこれが問題の最適解だろうか？

## 後半戦：750円以内で買う場合

<font color="red">次に、本来の問題「750円以内で買う（→15円以内で買う）」を考える。</font>さらに、解を一つに絞る設定も加えて完璧にする。

**＜条件式３（超強い条件）＞**<br>
ナップサック問題の鬼門がここ。応用D3の条件式をしっかり予習しておくこと。

合計金額が15以内になるということは、0, 1, 2, …, 15まで16個の整数のどれかになるということ。そのため補助ビットを16個用意するのが普通だが、もう少し節約できないか考える。合計金額が11以下のとき、おいしいみずを4円で買えばより良い解になる。したがって合計金額は12, 13, 14, 15の４通りしか取り得ない。

以上より、合計金額が「12, 13, 14, 15のどれかになる」ように補助ビットを４個使って設定する。かつ、この条件は条件式１よりもさらに優先しなければならないため<sup>[要解説]</sup>、超強い条件として係数10をかけることにする。<sup><font color="red">[係数1にすると何が起きるか、この先は君の目で確かめてくれ！]</font></sup>
```
#補助ビットを1つだけ1にする
#これにより (12*s0 + 13*s1 + 14*s2 + 15*s3) で 「12 or 13 or 14 or 15」 を表現できる
H += 10 * (s0 + s1 + s2 + s3 - 1)**2
```

これを利用して条件式１を微修正する。７個のドリンクをそれぞれ取るか取らないかして、合計金額を「12 or 13 or 14 or 15」にする。
```
H += (4*q0 + 4*q1 + 4*q2 + 6*q3 + 6*q4 + 7*q5 + 7*q6 - (12*s0 + 13*s1 + 14*s2 + 15*s3))**2
```

**＜条件式４（なくてもいいので強さはテキトー）＞**<br>
解が複数に分身して出てくるので、例えばおいしいみずを２本買うのであれば [1, 1, 0] のように降順のパターンだけが得られる設定をする。複数の量子ビットを降順にする応用C2を使用する。
```
H += (1 - q0) * q1
H += (1 - q1) * q2
```

### 後半戦：コード

In [5]:
from tytan import symbols, Compile, sampler
import numpy as np

#量子ビットを用意する
q0 = symbols('q0')
q1 = symbols('q1')
q2 = symbols('q2')
q3 = symbols('q3')
q4 = symbols('q4')
q5 = symbols('q5')
q6 = symbols('q6')

#補助ビットを用意する
s0 = symbols('s0')
s1 = symbols('s1')
s2 = symbols('s2')
s3 = symbols('s3')

#補助ビットを1つだけ1にする（超強い条件）
#これにより (12*s0 + 13*s1 + 14*s2 + 15*s3) で 「12 or 13 or 14 or 15」 を表現できる
H = 0
H += 10 * (s0 + s1 + s2 + s3 - 1)**2

#7個のドリンクをそれぞれ取るか取らないか、値段を係数にして合計が「12 or 13 or 14 or 15」になる（強い条件）
H += (4*q0 + 4*q1 + 4*q2 + 6*q3 + 6*q4 + 7*q5 + 7*q6 - (12*s0 + 13*s1 + 14*s2 + 15*s3))**2

#7個のドリンクをそれぞれ取るか取らないか、回復量を係数にして報酬とする（弱い条件）
H += -0.1 * (3*q0 + 3*q1 + 3*q2 + 5*q3 + 5*q4 + 7*q5 + 7*q6)

#おいしいみずを降順にする
H += (1 - q0) * q1
H += (1 - q1) * q2
#サイコソーダを降順にする
H += (1 - q3) * q4
#ミックスオレを降順にする
H += (1 - q5) * q6


#コンパイル
qubo, offset = Compile(H).get_qubo()
print(f'offset\n{offset}')

#サンプラー選択
solver = sampler.SASampler()

#サンプリング
result = solver.run(qubo, shots=1000)

#上位5件
for r in result:
    print(r)

offset
10
[{'q0': 0.0, 'q1': 0.0, 'q2': 0.0, 'q3': 0.0, 'q4': 0.0, 'q5': 1.0, 'q6': 1.0, 's0': 0.0, 's1': 0.0, 's2': 1.0, 's3': 0.0}, -11.399999999999977, 199]
[{'q0': 1.0, 'q1': 1.0, 'q2': 0.0, 'q3': 0.0, 'q4': 0.0, 'q5': 1.0, 'q6': 0.0, 's0': 0.0, 's1': 0.0, 's2': 0.0, 's3': 1.0}, -11.299999999999983, 82]
[{'q0': 0.0, 'q1': 0.0, 'q2': 0.0, 'q3': 1.0, 'q4': 0.0, 'q5': 1.0, 'q6': 0.0, 's0': 0.0, 's1': 1.0, 's2': 0.0, 's3': 0.0}, -11.199999999999989, 188]
[{'q0': 1.0, 'q1': 1.0, 'q2': 0.0, 'q3': 1.0, 'q4': 0.0, 'q5': 0.0, 'q6': 0.0, 's0': 0.0, 's1': 0.0, 's2': 1.0, 's3': 0.0}, -11.099999999999994, 217]
[{'q0': 0.0, 'q1': 0.0, 'q2': 0.0, 'q3': 1.0, 'q4': 1.0, 'q5': 0.0, 'q6': 0.0, 's0': 1.0, 's1': 0.0, 's2': 0.0, 's3': 0.0}, -11.0, 190]
[{'q0': 1.0, 'q1': 1.0, 'q2': 1.0, 'q3': 0.0, 'q4': 0.0, 'q5': 0.0, 'q6': 0.0, 's0': 1.0, 's1': 0.0, 's2': 0.0, 's3': 0.0}, -10.899999999999977, 124]


ベストな解が１通り出た。**みず０本、ソーダ０本、オレ２本**で、後ろの４つは補助ビットである。マイナスオフセットより1.4低いエネルギーなので回復量14に上がったことになる。**補助ビットの右から２つ目が立っている**ことから、合計金額は15ではなく14と分かる。

以上、みず０本、ソーダ０本、オレ２本で合計金額700円（残金50円）、合計回復量は140が最適解である。